**Introduction**

I previously built a XGboost to label data without context. Thought the results are good, but it's not satisfying.

And later I built a shallow RNN(due to the limitations of hardwares), and I found the RNN to be so disappoint.

Therefore, I built a new XGboost model to use the context to label data.

And most importantly, unlike the previous notebook, **this one is trained with all data, including PLAIN VERBATIM LETTERS etc.**

The major difference in the XGboost without context and with context is their dataset preparation.

XGboost with context(we will use XGBwithC for short) need the previous words(regardless of whether it comes from the same sentence or not). So we define a padding and 2 special symbols to let XGBoost understand, what's is the boundary of words and what are spaces that can be safely ignored.

**Results**

Both trained on 960,000 samples(words, not sentences) from en_train.csv and both have exactly the same parameters for training.

After 60 epochs, here are the total different results:

*XGBoost without context:

valid-merror:**0.005521**	train-merror:**0.004168***

*XGBoost with context:

valid-merror:**0.003729**	train-merror:**0.000811***

**Analysis**

I noticed astonishingly, XGBoost with context performs so well that it outperforms XGboost without context by a great margin. And what's more, **XGboost with context nearly perfectly fits to the training dataset it has seen.** I believe, if built larger and fed more data, it might be able to challenge deep RNN.

**Notebook explaintion**

Due to the time limit on notebook, I choose 320,000 samples instead of 960,000 to save time.

You can view outputs in this notebook.

**Final Words**

I have really limited time and I lack powerful computers(I only have my faithful alienware 15 r2 by my side)

Therefore I won't be participating in this competition any more.

And **I hope this script can help you in your research**.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split

/home/phil/anaconda2/envs/p35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 320000

out_path = r'.'

### Read in data

In [3]:
df = pd.read_csv('en_train.csv')


In [4]:
df.head(20)

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Brillantaisia,Brillantaisia
1,0,1,PLAIN,is,is
2,0,2,PLAIN,a,a
3,0,3,PLAIN,genus,genus
4,0,4,PLAIN,of,of
5,0,5,PLAIN,plant,plant
6,0,6,PLAIN,in,in
7,0,7,PLAIN,family,family
8,0,8,PLAIN,Acanthaceae,Acanthaceae
9,0,9,PUNCT,.,.


### Structure y data

In [5]:
y_data =  pd.factorize(df['class'])


In [6]:
y_data

(array([0, 0, 0, ..., 9, 0, 1]),
 Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
        'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
        'FRACTION', 'TELEPHONE', 'ADDRESS'],
       dtype='object'))

In [7]:
labels = y_data[1]
y_data = y_data[0]

In [8]:
y_data[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 4, 1, 0, 1, 0])

In [9]:
labels

Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
       'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
       'FRACTION', 'TELEPHONE', 'ADDRESS'],
      dtype='object')

In [10]:
gc.collect()


111

### Structure x data

Loop through each before value and convert each letter to list of np arrays where each value in an array is the hexadecimal unicode code point equivalent for each letter:

In [42]:
x_data = []

In [43]:
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

In [44]:
x_data[0]

array([ 66, 114, 105, 108, 108,  97, 110, 116,  97, 105])

In [50]:
def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    print("pre1: ",pre)
    pre = [pre for x in np.arange(pad_size)]
    print("pre2: ",pre)
    data_with_pre = pre + data + pre
    print("data_with_pre:", data_with_pre)
    neo_data = []
    print("flag: ", np.arange(len(data_with_pre) - pad_size * 2))
    for i in np.arange(len(data_with_pre) - pad_size * 2):
        row = []
        for x in data_with_pre[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

In [46]:
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]


In [ ]:
data

In [49]:
x_data = np.array(con`text_window_transform(x_data, pad_size))

pre1:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
pre2:  [array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]


ValueError: operands could not be broadcast together with shapes (1,10) (320000,34) 

In [14]:
gc.collect()

0

In [15]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [16]:
print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)



Total number of samples: 320000
Use:  320000


In [17]:
print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)

x_data sample:
[ -1   0   0   0   0   0   0   0   0   0   0  -1  66 114 105 108 108  97
 110 116  97 105  -1 105 115   0   0   0   0   0   0   0   0  -1]
y_data sample:
0
labels:
Index(['PLAIN', 'PUNCT', 'DATE', 'LETTERS', 'CARDINAL', 'VERBATIM', 'DECIMAL',
       'MEASURE', 'MONEY', 'ORDINAL', 'TIME', 'ELECTRONIC', 'DIGIT',
       'FRACTION', 'TELEPHONE', 'ADDRESS'],
      dtype='object')


In [18]:
x_train = x_data
y_train = y_data
gc.collect()

0

In [21]:
x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2000)

gc.collect()

85

In [22]:
num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

In [23]:
param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}

In [24]:
model = xgb.train(param, dtrain, 50, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)
gc.collect()

[0]	valid-merror:0.008031	train-merror:0.007924
Multiple eval metrics have been passed: 'train-merror' will be used for early stopping.

Will train until train-merror hasn't improved in 20 rounds.
[10]	valid-merror:0.005469	train-merror:0.003451
[20]	valid-merror:0.005094	train-merror:0.001965
[30]	valid-merror:0.004719	train-merror:0.001017
[40]	valid-merror:0.004688	train-merror:0.000451


35

In [25]:
# Will train until train-merror hasn't improved in 20 rounds.
# [10]	valid-merror:0.005469	train-merror:0.003535
# [20]	valid-merror:0.004938	train-merror:0.002073
# [30]	valid-merror:0.00475	train-merror:0.001031
# [40]	valid-merror:0.004625	train-merror:0.000493

In [26]:
pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

0

In [35]:
y_valid[0:10]

['PLAIN',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'DATE',
 'PLAIN',
 'PLAIN']

In [27]:
df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

In [28]:
df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

In [29]:
model.save_model(os.path.join(out_path, 'xgb_model'))